In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [3]:
metadata = pd.read_csv("C:\Maya\CS\AnomalyDetectionMicrobiome\Human_Microbiome_Project_1_6_2017_metadata.tab",sep='\t')

In [4]:
metadata.head()

,#SampleID,BarcodeSequence,LinkerPrimerSequence,run_prefix,body_habitat,body_product,body_site,bodysite,dna_extracted,elevation,...,sample_type,scientific_name,sequencecenter,taxon_id,title,qiita_study_title,qiita_study_alias,qiita_owner,qiita_principal_investigator,Description
0,1927.SRS055168.SRX020678.SRR048038,TCAGACTCCTC,ATTACCGCGGCTGCTGG,SRR048038,UBERON:skin,UBERON:sebum,UBERON:skin of elbow,R_Antecubital_fossa,True,97.0,...,Skin,human skin metagenome,BCM,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS055168.SRX020678.SRR048038
1,1927.SRS015346.SRX020577.SRR044088,TCAGAACCAGC,ATTACCGCGGCTGCTGG,SRR044088,UBERON:vagina,UBERON:mucus,UBERON:central vagina,Mid_vagina,True,97.0,...,Vaginal,human vaginal metagenome,WUGSC,1632839,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS015346.SRX020577.SRR044088
2,1927.SRS021613.SRX020675.SRR047757,TCAGAGCTTC,ATTACCGCGGCTGCTGG,SRR047757,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,Tongue_dorsum,True,97.0,...,Oral,human oral metagenome,BCM,447426,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS021613.SRX020675.SRR047757
3,1927.SRS022512.SRX020678.SRR048023,TCAGTATCAAC,ATTACCGCGGCTGCTGG,SRR048023,UBERON:skin,UBERON:sebum,UBERON:skin of elbow,L_Antecubital_fossa,True,97.0,...,Skin,human skin metagenome,BCM,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS022512.SRX020678.SRR048023
4,1927.SRS016305.SRX020525.SRR044526,TCAGCCACGGTC,ATTACCGCGGCTGCTGG,SRR044526,UBERON:skin,UBERON:sebum,UBERON:skin of external ear,L_Retroauricular_crease,True,97.0,...,Skin,human skin metagenome,WUGSC,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS016305.SRX020525.SRR044526


In [5]:
data = pd.read_csv("C:\Maya\CS\AnomalyDetectionMicrobiome\Human_Microbiome_Project_1_6_2017_data.tab",sep='\t')

In [6]:
data.head()

,#OTU ID,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,...,1928.SRS042698.SRX020530.SRR045847,1928.SRS023827.SRX021470.SRR052699,1928.SRS022769.SRX022226.SRR058089,1928.SRS024716.SRX021470.SRR052699,1928.SRS045191.SRX021470.SRR052699,1928.SRS045171.SRX021470.SRR052699,1928.SRS024704.SRX021470.SRR052699,1928.SRS020726.SRX022239.SRR058109,1928.SRS022067.SRX022231.SRR058120,1928.SRS023274.SRX022223.SRR058086
0,4376890,6.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,188753,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,114076,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,253429,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4467992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### extract relevant samples

In [7]:
filtered_data_with_OTUs = data.loc[:, data.sum() > 3000] #5728 samples, 13268 features
filtered_data = filtered_data_with_OTUs.iloc[:, 1:]
filtered_data

,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,1927.SRS014913.SRX020546.SRR043697,...,1928.SRS042246.SRX020686.SRR048627,1928.SRS057075.SRX022222.SRR058085,1928.SRS043338.SRX022227.SRR058090,1928.SRS022777.SRX022226.SRR058089,1928.SRS016730.SRX019682.SRR040947,1928.SRS021283.SRX022235.SRR058095,1928.SRS018288.SRX019687.SRR041227,1928.SRS016559.SRX019636.SRR040605,1928.SRS016559.SRX019635.SRR040662,1928.SRS018292.SRX019687.SRR041262
0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### relative abundance - calc sum

In [8]:
final_data = filtered_data.copy()

for colname in filtered_data.columns:
    final_data.loc[:,colname] = filtered_data[colname] / sum(filtered_data[colname])

final_data.head()

,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,1927.SRS014913.SRX020546.SRR043697,...,1928.SRS042246.SRX020686.SRR048627,1928.SRS057075.SRX022222.SRR058085,1928.SRS043338.SRX022227.SRR058090,1928.SRS022777.SRX022226.SRR058089,1928.SRS016730.SRX019682.SRR040947,1928.SRS021283.SRX022235.SRR058095,1928.SRS018288.SRX019687.SRR041227,1928.SRS016559.SRX019636.SRR040605,1928.SRS016559.SRX019635.SRR040662,1928.SRS018292.SRX019687.SRR041262
0,0.001055,0.001894,0.000218,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000047,0.000213,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000248,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001515,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000164,0.00024,0.000265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Isolation Forest
data for IF

In [31]:
all_feces = metadata[metadata['body_site'] == 'UBERON:feces']['#SampleID']
feces = [sample for sample in all_feces if sample in final_data.columns]
feces_data = final_data[feces] #418 samples

#all _body sites except of feces
all_body_sites = metadata[metadata['body_site'] != 'UBERON:feces' ]['body_site'].drop_duplicates()
print(all_body_sites)

0                    UBERON:skin of elbow
1                   UBERON:central vagina
2                           UBERON:tongue
4             UBERON:skin of external ear
5                            UBERON:mouth
6                            UBERON:tooth
8                  UBERON:palatine tonsil
11                   UBERON:buccal mucosa
14               UBERON:vaginal introitus
17                          UBERON:throat
18                         UBERON:gingiva
19                  UBERON:anterior nares
20                     UBERON:hard palate
64      UBERON:posterior fornix of vagina
3534                  UBERON:skin of nose
3537                        UBERON:saliva
Name: body_site, dtype: object


In [43]:
all_auc = []

for body_site in all_body_sites:
    now_auc = []
    i = 0
    while i < 50:
        all_body_site = metadata[metadata['body_site'] == body_site]['#SampleID']
        final_all_body_site = [sample for sample in all_body_site if sample in  final_data.columns]
        all_body_site_data = final_data[final_all_body_site]
        if len(all_body_site_data.iloc[0]) >= 42:
            few_all_body_site_data = all_body_site_data.iloc[: ,-42:] #42 samples
            more_res = [0 for i in range(42)]
            
             #################choose random 42 ###############
            
        else:
            few_all_body_site_data = all_body_site_data
            more_res = [0 for i in range(len(few_all_body_site_data.iloc[0]))]
            
        results = [1 for i in range(418)]
        
        y = results + more_res

        inputIF = pd.concat([feces_data, few_all_body_site_data], axis=1) #42 samples out of 460 should be outliers

        model = IsolationForest(n_estimators=100)
        model.fit(inputIF.transpose())  # fit 100 trees

        scores = model.decision_function(inputIF.transpose()) #higher score means more normal
        #sortedScores = sorted(scores)
        # indices of anomalies (body site != feces) in sorted scores. higher score = higher indices = more normal !
        #anomalies_indices_in_sorted = [i for i in range(len(sortedScores)) if sortedScores[i] in scores[-42:]]
        #print(anomalies_indices_in_sorted) # if algorithm is working ok, indices are expected to be low. in reality we get differernt indices, some are very high. 
        
        # calculate area under the curve
        now_auc.append(roc_auc_score(y, scores))
        i += 1
        
    all_auc.append(now_auc)
        
print(all_auc)

all_auc = [[0.616142629300524, 0.6383572567783093, 0.5963773069036227, 0.6338004101161995, 0.6377306903622694, 0.603611300979722, 0.5831624515835042, 0.6329175210754159, 0.7038049669628617, 0.5924755069491912, 0.6768626110731375, 0.5813966735019367, 0.6433128275233537, 0.6401230348598769, 0.6944064707222602, 0.6308384597858283, 0.6263385737069946, 0.5777511961722488, 0.6328320802005012, 0.6513442697653224, 0.6728753702437912, 0.6473000683526999, 0.5798587377534746, 0.5986557302346777, 0.5558213716108454, 0.6254272043745729, 0.6701982228298018, 0.7304055593529278, 0.6471861471861471, 0.6431989063568011, 0.7268740031897927, 0.6901913875598086, 0.5944976076555024, 0.697425381635908, 0.6835839598997494, 0.617680565048986, 0.7439052175894281, 0.6066871724766462, 0.5787764866712235, 0.5733082706766917, 0.6071428571428572, 0.6940077466393255, 0.578548644338118, 0.5864376851218956, 0.6512303485987696, 0.6529961266803372, 0.5143540669856459, 0.5881180223285487, 0.6483253588516746, 0.6507177033492824], [0.12141148325358851, 0.12494303941672366, 0.15789473684210525, 0.10042150831624516, 0.12263613579403057, 0.07379243563454091, 0.16547049441786285, 0.15880610617452723, 0.08965595807701071, 0.12152540442014127, 0.10856687172476648, 0.11506037821827296, 0.09822852586010482, 0.13793005240373662, 0.16569833675096837, 0.2314023695602643, 0.13727500569605833, 0.18571998177261334, 0.13189223057644112, 0.12012987012987016, 0.1386135794030531, 0.11335156071998179, 0.20243791296422875, 0.12388926862611073, 0.11369332421964004, 0.15017657780815677, 0.10608908635224426, 0.12642401458190933, 0.14035087719298245, 0.13915470494417864, 0.13140806561859195, 0.16410344041922992, 0.16250854408749146, 0.09022556390977443, 0.14205969469127364, 0.1494930508088403, 0.10084871269081797, 0.13738892686261106, 0.15117338801549327, 0.114149008885851, 0.12263613579403054, 0.12050011392116655, 0.15974595579858739, 0.13308840282524492, 0.19110275689223058, 0.08153907496012759, 0.11095921622237413, 0.1983937115516063, 0.19135907951697428, 0.09865573023467761], [0.4001480975165186, 0.406015037593985, 0.27665755297334244, 0.6065162907268171, 0.3470608339029392, 0.4154704944178629, 0.4258373205741627, 0.33014354066985646, 0.3967874231032126, 0.38345864661654133, 0.4993164730006835, 0.5464228753702438, 0.5902255639097744, 0.3761676919571656, 0.351617680565049, 0.6069150148097516, 0.5413533834586466, 0.47459557985873774, 0.4592731829573935, 0.4172932330827067, 0.41057188425609487, 0.3601617680565049, 0.33595352016404645, 0.4699248120300752, 0.5375939849624061, 0.5422647527910687, 0.4616085668717248, 0.3685919343814081, 0.39786967418546365, 0.38704716336295286, 0.4156983367509683, 0.5926748689906585, 0.6587491455912509, 0.3852244246981089, 0.36306675780359987, 0.5766689450899978, 0.5365686944634314, 0.39171793119161535, 0.44076099339257235, 0.45676691729323304, 0.4765322396901344, 0.32946001367054, 0.42116655274550013, 0.4336978810663021, 0.55622009569378, 0.4781840966051492, 0.5484164957849168, 0.36631351105035315, 0.4486785144679882, 0.3455798587377535], [0.37149692412850305, 0.26144907723855093, 0.29710640236956026, 0.28354978354978355, 0.25911369332421963, 0.2978468899521531, 0.3635794030530873, 0.3251310093415356, 0.23108908635224426, 0.44594440647072225, 0.36226930963773074, 0.30767259056732743, 0.2768284347231716, 0.33438710412394623, 0.2446172248803828, 0.24948735475051267, 0.20625427204374575, 0.28565732513100933, 0.31655844155844154, 0.2323137388926863, 0.3093814080656186, 0.21878560036454775, 0.39684438368648894, 0.28565732513100933, 0.32111528822055135, 0.27392344497607657, 0.2897015265436318, 0.2550125313283208, 0.2983025746183641, 0.23524720893141948, 0.23476304397357028, 0.199419002050581, 0.2774550011392117, 0.29892914103440416, 0.30451127819548873, 0.3212576896787423, 0.36426293005240373, 0.2845181134654819, 0.3364092048302575, 0.2699362041467305, 0.27435064935064934, 0.30980861244019137, 0.3807814992025519, 0.2760309865573024, 0.27870813397129185, 0.32396331738437, 0.30923900660742765, 0.2641831852358168, 0.27625882889040787, 0.2972488038277512], [0.59563681932103, 0.6668945089997721, 0.4877249943039417, 0.5193096377306904, 0.5034176349965824, 0.6207564365459103, 0.5433470038733197, 0.47114946457051715, 0.4752221462747779, 0.42720437457279564, 0.5280815675552518, 0.49962975620870353, 0.49988607883344727, 0.702323991797676, 0.566928685349738, 0.6258259284575074, 0.5502392344497608, 0.5849851902483482, 0.5497265892002734, 0.6576099339257233, 0.4838231943495101, 0.46610845295055825, 0.4609820004556846, 0.6011619958988381, 0.5361130097972203, 0.6249145591250854, 0.6001367053998633, 0.7082194121667805, 0.5466791979949874, 0.4425267714741399, 0.5710583276372748, 0.5292207792207793, 0.4909432672590567, 0.5037593984962406, 0.4090909090909091, 0.5942697653223967, 0.4632604237867396, 0.5335497835497836, 0.4665641376167692, 0.4039644565960355, 0.5670426065162907, 0.5185691501480976, 0.6991911597174754, 0.5239519252677147, 0.5593529277739804, 0.47305764411027573, 0.5156641604010025, 0.5878332194121668, 0.5298188653451812, 0.5477329687856004], [0.6700843016632492, 0.6241171109592163, 0.5897698792435635, 0.6941786283891547, 0.5518341307814991, 0.7087035771246296, 0.6847801321485532, 0.632490316700843, 0.585554796081112, 0.7081909318751424, 0.6887104123946229, 0.6604579630895421, 0.6200159489633175, 0.6593757120072908, 0.6609136477557531, 0.5795739348370927, 0.6488380041011621, 0.5256322624743678, 0.592162223741171, 0.6713374344953292, 0.6368762816131237, 0.6286739576213259, 0.6078263841421736, 0.6054910002278423, 0.6466165413533834, 0.6003075871496925, 0.6438254727728411, 0.6523125996810207, 0.6019024834814309, 0.7112098427887901, 0.773524720893142, 0.6384711779448622, 0.5285942128047392, 0.7079630895420369, 0.5929596719070402, 0.7200387331966279, 0.6504329004329005, 0.6560719981772613, 0.6458760537707906, 0.7572909546593757, 0.5745044429254955, 0.6699703804966963, 0.5907951697425381, 0.7167919799498746, 0.5989974937343358, 0.5793460925039873, 0.6208133971291867, 0.5685805422647529, 0.7729551150603782, 0.6590909090909092], [0.6484962406015038, 0.6293574846206427, 0.6231487810435179, 0.5853269537480064, 0.6270221007063113, 0.5737639553429026, 0.5543973570289359, 0.5858965595807702, 0.4941330599225336, 0.6303827751196173, 0.5837890179995443, 0.6209273182957393, 0.5734791524265209, 0.578092959671907, 0.4479380268853953, 0.6392686261107314, 0.5441729323308271, 0.5738209159261791, 0.43483709273182963, 0.6014467988152198, 0.4784688995215311, 0.5459102301207565, 0.6326042378673957, 0.6130667578035999, 0.5975734791524265, 0.6094212804739121, 0.588174982911825, 0.6012759170653907, 0.5735930735930737, 0.5289074960127591, 0.587092731829574, 0.6060036454773297, 0.533777625882889, 0.5848712690817954, 0.577010708589656, 0.41960013670539986, 0.697881066302119, 0.4889496468443837, 0.606516290726817, 0.5421508316245157, 0.5310435178856232, 0.5273980405559353, 0.5837320574162679, 0.5059808612440191, 0.49624060150375937, 0.5813397129186603, 0.6576099339257233, 0.5856687172476646, 0.44736842105263164, 0.607142857142857], [0.49316473000683525, 0.550580997949419, 0.4266347687400319, 0.5134426976532239, 0.568010936431989, 0.4607541581225792, 0.38402825244930505, 0.4580200501253133, 0.5243791296422875, 0.5787195260879472, 0.5756436545910231, 0.42484051036682613, 0.5630553656869446, 0.6105320118478013, 0.44668489405331513, 0.538049669628617, 0.4677318295739349, 0.5670995670995671, 0.5819662793347002, 0.3948507632718159, 0.5672134882661197, 0.5364547732968785, 0.4821713374344954, 0.507632718159034, 0.5663875598086124, 0.4458874458874459, 0.5011961722488039, 0.5997379813169286, 0.4783549783549783, 0.5405274550011392, 0.4778708133971292, 0.5089997721576669, 0.5395021645021644, 0.530872636135794, 0.6173957621326043, 0.45169742538163593, 0.5341763499658237, 0.36933242196400096, 0.48547505126452495, 0.5983709273182958, 0.5356573251310094, 0.49248120300751885, 0.507632718159034, 0.46793119161540214, 0.6258259284575074, 0.6770334928229665, 0.4392800182273866, 0.6450786056049215, 0.4839371155160629, 0.5182273866484393], [0.15567327409432674, 0.18213146502620184, 0.13998063340168604, 0.1764069264069264, 0.2532467532467533, 0.17603668261563, 0.1251993620414673, 0.1923274094326726, 0.1877420824789246, 0.14433811802232854, 0.2807871952608795, 0.20064365459102304, 0.1179084073820916, 0.19434951013898383, 0.1404647983595352, 0.16689450899977215, 0.18586238323080428, 0.1988493962178173, 0.14456596035543406, 0.15860674413305992, 0.1730177717019822, 0.16660970608339032, 0.26896787423103213, 0.2502278423331055, 0.16398951925267716, 0.18648894964684437, 0.15669856459330145, 0.20186830713146503, 0.18267259056732743, 0.08942811574390522, 0.18258714969241285, 0.2363864205969469, 0.21915584415584413, 0.16296422875370245, 0.18449532923217138, 0.25079744816586924, 0.1482683982683983, 0.14035087719298248, 0.14408179539758487, 0.24567099567099568, 0.17321713374344955, 0.15521758942811578, 0.17170767828662564, 0.191928685349738, 0.14325586694007747, 0.11181362497151971, 0.21360218728639782, 0.18910913647755753, 0.15749601275917063, 0.11881977671451356], [0.6369902027796764, 0.6499202551834131, 0.5702893597630441, 0.3456083390293917, 0.6179084073820915, 0.5918774208247892, 0.5742766005923902, 0.5668147641831852, 0.5249487354750513, 0.567555251765778, 0.4899749373433584, 0.552118933697881, 0.4249829118250171, 0.5326953748006379, 0.4820004556846662, 0.5836181362497153, 0.44765322396901347, 0.4344953292321714, 0.4991455912508544, 0.5148667122351334, 0.5328092959671906, 0.5645933014354066, 0.548758259284575, 0.47419685577580317, 0.45933014354066987, 0.5557644110275689, 0.47715880610617456, 0.5475620870357711, 0.4656527682843472, 0.6016746411483254, 0.59107997265892, 0.6331168831168831, 0.6059466848940533, 0.6293005240373662, 0.48165869218500795, 0.5940419229892914, 0.7367281840966051, 0.5055821371610845, 0.456596035543404, 0.5008544087491457, 0.5698336750968329, 0.456596035543404, 0.5380496696286169, 0.6659831396673502, 0.48325358851674644, 0.40498974709501023, 0.3873604465709729, 0.5517771701982227, 0.5986557302346776, 0.5637958532695374], [0.2799043062200957, 0.3038277511961722, 0.270619731146047, 0.30137844611528825, 0.4616655274550011, 0.3239348370927319, 0.384313055365687, 0.3122009569377991, 0.48798131692868535, 0.38590795169742537, 0.4152996126680337, 0.3508771929824561, 0.4718329915698337, 0.2382661198450672, 0.33347573479152426, 0.24111414900888584, 0.3147072226019595, 0.3224823422191843, 0.4504442925495557, 0.35127591706539074, 0.4331282752335384, 0.3691615402141718, 0.3697596263385737, 0.4019138755980861, 0.4592162223741172, 0.24883230804283438, 0.28406242879927096, 0.2976475279106858, 0.3376908179539758, 0.4972374117110958, 0.3071884256094783, 0.30217589428115743, 0.40686944634313055, 0.4719753930280246, 0.499088630667578, 0.3507063112326271, 0.41615402141717933, 0.34666210982000456, 0.4481089086352244, 0.2706766917293233, 0.4511278195488722, 0.3468899521531101, 0.3741740715424926, 0.29784688995215314, 0.28918888129414444, 0.4226475279106858, 0.33373205741626794, 0.3157894736842105, 0.3385737069947597, 0.2888755980861245], [0.27905540978546073, 0.32532026547306686, 0.3694242938725112, 0.34820188300663685, 0.33215002315172093, 0.3250501620620466, 0.3067603025158203, 0.3149405772495756, 0.33747491896897674, 0.328985954622627, 0.27172403148634044, 0.34195091835159747, 0.3732829140299429, 0.32304367958018215, 0.3394042290476926, 0.3184905077944127, 0.26805834233678033, 0.3830838092298194, 0.2564438956629109, 0.4292329063127026, 0.2904769254514586, 0.2594536193857077, 0.35468436487112215, 0.394968359314709, 0.28237382312085196, 0.37991974070072543, 0.34272264238308375, 0.29016823583886403, 0.2722256521068066, 0.29425837320574166, 0.3357771260997067, 0.319493749035345, 0.4389566291094305, 0.38921901528013586, 0.2402762772032721, 0.2296650717703349, 0.2995060966198487, 0.3238925760148171, 0.34642691773421824, 0.3881000154344806, 0.3448834696712456, 0.3809229819416577, 0.2677496527241858, 0.2857308226578176, 0.3631347430158975, 0.2458712764315481, 0.24062355301744096, 0.33060657508874824, 0.3891804290785615, 0.29746102793641], [0.4600136705399864, 0.5666438824333561, 0.4612098427887902, 0.42543859649122806, 0.5349168375484166, 0.44036226930963773, 0.6377876509455456, 0.6404078377762588, 0.558384597858282, 0.5401002506265664, 0.481203007518797, 0.37451583504215086, 0.5539986329460013, 0.46274777853725224, 0.5369674185463659, 0.5592959671907041, 0.48752563226247436, 0.5583276372750057, 0.5320118478013216, 0.6446798815219869, 0.5612326270221007, 0.459045340624288, 0.6504329004329004, 0.571827295511506, 0.4027113237639553, 0.6236044657097288, 0.45494417862838915, 0.5694349510138983, 0.5464798359535201, 0.6289587605377079, 0.4568808384597858, 0.5794030530872637, 0.6136933242196402, 0.521701982228298, 0.43962178172704497, 0.4461722488038277, 0.5378787878787878, 0.4797220323536113, 0.6273638642059696, 0.5747892458418775, 0.5666438824333561, 0.5775233538391434, 0.6008202323991798, 0.5811688311688311, 0.5220437457279562, 0.6579516974253816, 0.6393255866940079, 0.5200501253132832, 0.5845295055821371, 0.5565618591934381], [0.08199475962633859, 0.06561859193438141, 0.08387445887445888, 0.04192298929141036, 0.08837434495329233, 0.09691843244474822, 0.055422647527910686, 0.05738778765094555, 0.10042150831624518, 0.04243563454089771, 0.08583959899749374, 0.06009341535657325, 0.12030075187969926, 0.05223285486443381, 0.05724538619275462, 0.072738664843928, 0.07678286625655045, 0.11386420596946915, 0.08498519024834815, 0.08689336978810665, 0.07091592617908409, 0.07177033492822968, 0.05844155844155844, 0.0800865800865801, 0.07450444292549557, 0.08908635224424699, 0.09483937115516064, 0.029733424470266584, 0.07931761221234905, 0.06351105035315563, 0.055080884028252446, 0.04536910457963089, 0.07399179767600822, 0.09261790840738211, 0.09327295511506038, 0.09498177261335157, 0.07040328092959673, 0.12756322624743677, 0.09079516974253816, 0.04796081111870586, 0.0873775347459558, 0.0938995215311005, 0.05206197311460469, 0.03232513100934154, 0.03520164046479836, 0.12531328320802007, 0.06567555251765778, 0.03722374117110959, 0.08182387787650945, 0.10378218272955114], [0.26953748006379585, 0.3920881749829118, 0.300580997949419, 0.3898382319434951, 0.30556504898610165, 0.4469127363864206, 0.30362838915470497, 0.4069833675096833, 0.2756892230576441, 0.32840624287992704, 0.3494531784005469, 0.34637730690362273, 0.41048644338118023, 0.35415242652084755, 0.36562998405103664, 0.39462292093871043, 0.4679596719070403, 0.377136021872864, 0.4294827979038506, 0.326697425381636, 0.39154704944178625, 0.37810435178856233, 0.4398496240601503, 0.32595693779904306, 0.2971633629528367, 0.31473570289359765, 0.30379927090453407, 0.37497151970836184, 0.3904078377762588, 0.4662793347003873, 0.4053599908863067, 0.3178970152654363, 0.20457393483709274, 0.36719640009113697, 0.35173160173160173, 0.41541353383458646, 0.30291638186375025, 0.28605604921394395, 0.4162109820004557, 0.3427318295739349, 0.3346149464570517, 0.3957051720209615, 0.2677147413989519, 0.2461266803372067, 0.3849396217817271, 0.3478582820688084, 0.3684210526315789, 0.501680337206653, 0.34398496240601506, 0.4516404647983595], [0.626224652540442, 0.7263043973570289, 0.6862611073137389, 0.6939507860560492, 0.6042378673957621, 0.647015265436318, 0.7920369104579631, 0.6727614490772386, 0.8198906356801093, 0.5862952836637046, 0.7089314194577352, 0.7542150831624514, 0.6987924356345409, 0.6605718842560948, 0.7038049669628617, 0.6143198906356802, 0.7285828206880838, 0.721120984278879, 0.7468102073365231, 0.7109250398724083, 0.7255639097744361, 0.7299498746867168, 0.6354522670312144, 0.7128047391205286, 0.5809979494190021, 0.849794941900205, 0.5356573251310094, 0.7403167008430166, 0.574105718842561, 0.7240829346092504, 0.6086807928913193, 0.6826725905673274, 0.7253360674413306, 0.6642173615857826, 0.672077922077922, 0.7211779448621555, 0.6126110731373889, 0.691786283891547, 0.6435406698564593, 0.6802802460697197, 0.5446001367053999, 0.6672362724994305, 0.8397129186602872, 0.7259056732740944, 0.7323991797676007, 0.5692071086807929, 0.7345067213488267, 0.729209387104124, 0.5462519936204147, 0.5644793802688539]]

[[0.616142629300524, 0.6383572567783093, 0.5963773069036227, 0.6338004101161995, 0.6377306903622694, 0.603611300979722, 0.5831624515835042, 0.6329175210754159, 0.7038049669628617, 0.5924755069491912, 0.6768626110731375, 0.5813966735019367, 0.6433128275233537, 0.6401230348598769, 0.6944064707222602, 0.6308384597858283, 0.6263385737069946, 0.5777511961722488, 0.6328320802005012, 0.6513442697653224, 0.6728753702437912, 0.6473000683526999, 0.5798587377534746, 0.5986557302346777, 0.5558213716108454, 0.6254272043745729, 0.6701982228298018, 0.7304055593529278, 0.6471861471861471, 0.6431989063568011, 0.7268740031897927, 0.6901913875598086, 0.5944976076555024, 0.697425381635908, 0.6835839598997494, 0.617680565048986, 0.7439052175894281, 0.6066871724766462, 0.5787764866712235, 0.5733082706766917, 0.6071428571428572, 0.6940077466393255, 0.578548644338118, 0.5864376851218956, 0.6512303485987696, 0.6529961266803372, 0.5143540669856459, 0.5881180223285487, 0.6483253588516746, 0.6507177033492824], [0

In [38]:
all_feces = metadata[metadata['body_site'] == 'UBERON:feces']['#SampleID']
all_skin_of_nose = metadata[metadata['body_site'] == 'UBERON:skin of nose']['#SampleID']
feces = [sample for sample in all_feces if sample in final_data.columns]
skin_of_nose = [sample for sample in all_skin_of_nose if sample in  final_data.columns]


feces_data = final_data[feces] #418 samples

skin_of_nose_data = final_data[skin_of_nose] #203 samples

few_skin_of_nose_data = skin_of_nose_data.iloc[: ,-42:] #42 samples

inputIF = pd.concat([feces_data, few_skin_of_nose_data], axis=1) #42 samples out of 460 should be outliers

13268


### IF model

In [26]:
model = IsolationForest(n_estimators=100)
model.fit(inputIF.transpose())  # fit 100 trees
 
scores = model.decision_function(inputIF.transpose()) #higher score means more normal
sortedScores = sorted(scores)

# indices of anomalies (skin of nose) in sorted scores. higher score = higher indices = more normal !
anomalies_indices_in_sorted = [i for i in range(len(sortedScores)) if sortedScores[i] in scores[-42:]]
print(anomalies_indices_in_sorted) # if algorithm is working ok, indices are expected to be low. in reality we get differernt indices, some are very high. 

results = [1 for i in range(418)]
more_res = [0 for i in range(42)]
y = results + more_res

# calculate area under the curve
roc_auc_score(y, scores) #around 0.35


[1, 11, 66, 68, 85, 86, 120, 127, 128, 149, 158, 177, 179, 190, 192, 194, 225, 266, 274, 296, 318, 322, 327, 330, 335, 336, 345, 354, 359, 371, 372, 373, 377, 382, 389, 395, 398, 419, 428, 430, 446, 447, 448]


ValueError: Found input variables with inconsistent numbers of samples: [460, 449]

graph depth of outlier vs normal
mann whitney test SCIPY
different body sites
bootstrap